# Confronto Insertion-sort e heap sort

In questo notebook verranno analizzati e confrontati i due algoritmi di ordinamento *insertion-sort* e *heap-sort. 

## Studio prestazioni attese

Prima di procedere con l'implementazione degli algoritmi è opportuno fare delle assunzioni sulle prestazioni che ci aspettiamo. 

### Insertion sort

L'insertion sort è un algoritmo di ordinamento che ordina **sul posto** ossia opera sull'array da ordinare senza aver bisogno di memoria aggiuntiva. 

L'insertion sort opera ordinando gradualmente un sottoarray sinistro dell'input fornito, incrementando di 1 la grandezza del sottoarray ad ogni iterazione fino a quando l'intero array non è ordinato. 

Possiamo quindi dedurre che nel **caso migliore**, ossia quando l'input è un array ordinato, l'insertion sort analizzera ogni elemento dell'array fino alla fine senza operare alcun cambiamento e dunque avremo un **costo lineare $O(n)$**. 

Nel **caso peggiore** ossia con in input un array ordinato al contrario l'array avrà un **costo quadratico $O(n^2)$**. 

Anche nel **caso medio** il costo sarà **quadratico $O(n^2)$**.